<a href="https://colab.research.google.com/github/Andre-Williams22/Reinforcement-Learning-Agent/blob/volatile-stock/most_volatile_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install alpaca_trade_api

In [ ]:
!pip3 install yfinance

In [33]:
import requests
import json as json
from datetime import datetime
import alpaca_trade_api as t
import logging
import pytz
import time
import csv
# input : year(date), num of stocks
# output: list of volatile stocks

import yfinance as yf
# from pandas_datareader import data as pdr
import pandas as pd


In [27]:
# # list all stocks
def get_stock_symbols():
  url = "ftp://ftp.nasdaqtrader.com/SymbolDirectory/nasdaqlisted.txt"
  df = pd.read_csv(url, sep="|")
  #convert df.symbol to lists of symbols
  symbols = df['Symbol'].tolist()
  return symbols

In [29]:
# time get stock symbols function
start = time.perf_counter()
print(get_stock_symbols())
end = time.perf_counter()
print("time", end - start)

['AACG', 'AACQ', 'AACQU', 'AACQW', 'AAL', 'AAME', 'AAOI', 'AAON', 'AAPL', 'AAWW', 'AAXJ', 'ABCB', 'ABCL', 'ABCM', 'ABEO', 'ABGI', 'ABIO', 'ABMD', 'ABNB', 'ABST', 'ABTX', 'ABUS', 'ACAC', 'ACACU', 'ACACW', 'ACAD', 'ACAHU', 'ACBI', 'ACCD', 'ACER', 'ACET', 'ACEV', 'ACEVU', 'ACEVW', 'ACGL', 'ACGLO', 'ACGLP', 'ACHC', 'ACHV', 'ACIU', 'ACIW', 'ACKIT', 'ACKIU', 'ACKIW', 'ACLS', 'ACMR', 'ACNB', 'ACOR', 'ACQRU', 'ACRS', 'ACRX', 'ACST', 'ACTC', 'ACTCU', 'ACTCW', 'ACTG', 'ACWI', 'ACWX', 'ADAG', 'ADAP', 'ADBE', 'ADER', 'ADERU', 'ADERW', 'ADES', 'ADI', 'ADIL', 'ADILW', 'ADMA', 'ADMP', 'ADMS', 'ADN', 'ADNWW', 'ADOC', 'ADOCR', 'ADOCW', 'ADP', 'ADPT', 'ADRE', 'ADSK', 'ADTN', 'ADTX', 'ADUS', 'ADV', 'ADVM', 'ADVWW', 'ADXN', 'ADXS', 'AEAC', 'AEACU', 'AEACW', 'AEGN', 'AEHL', 'AEHR', 'AEI', 'AEIS', 'AEMD', 'AEP', 'AEPPL', 'AEPPZ', 'AERI', 'AESE', 'AEY', 'AEYE', 'AEZS', 'AFBI', 'AFIB', 'AFIN', 'AFINO', 'AFINP', 'AFMD', 'AFRM', 'AFYA', 'AGBA', 'AGBAR', 'AGBAU', 'AGBAW', 'AGC', 'AGCUU', 'AGCWW', 'AGEN', 'AGFS',

In [30]:
def df_lookup(df, key_row, key_col):
  try:
    return df.iloc[key_row][key_col]
  except IndexError:
    return 0

In [24]:
def get_movement_list(df, period):
  movement_list = []
  symbols = get_stock_symbols()
  for stock in symbols:
    # get history
    curr_stock = yf.Ticker(stock)
    hist = curr_stock.history(period = period) #lookback 1 day

    low = float(10000)
    high = float(0)
    # print(curr_stock.info)
    for day in hist.itertuples(index=True, name='Pandas'):
      if day.Low < low:
        low = day.Low
      if high < day.High:
        high = day.High
    #for zero division error handling
    if low == 0:
      delta_percent = 0
    else:
      delta_percent = 100 * (high - low) / low #check for division by 0
    Open = df_lookup(hist, 0, "Open")

    # some error handling: 
    if len(hist >= 5):
      Close = df_lookup(hist, 4, "Close")
    else :
      Close = Open

    if (Open == 0): #correct zero division error
      delta_price = 0
    else:
      delta_price = 100 * (Close - Open) / Open 
 
    print(stock+" "+str(delta_percent)+ " "+ str(delta_price))
    pair = [stock, delta_percent, delta_price]
    movement_list.append(pair)
  return movement_list   

In [41]:
#test
# with open('employee_file.csv', mode='w') as employee_file:
#     employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

#     employee_writer.writerow(['John Smith', 'Accounting', 'November'])
#     employee_writer.writerow(['Erica Meyers', 'IT', 'March'])

def get_movement_list(df, period):
  movement_list = []
  f = open("output.csv", "w")
  stock_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  stock_writer.writerow(["stock", "delta_percent", "delta_price"]) # add header to csv
  symbols = ["AAPL", "MSFT"]
  for stock in symbols:
    # get history
    curr_stock = yf.Ticker(stock)
    hist = curr_stock.history(period = period) #lookback 1 day

    low = float(10000)
    high = float(0)
    # print(curr_stock.info)
    for day in hist.itertuples(index=True, name='Pandas'):
      if day.Low < low:
        low = day.Low
      if high < day.High:
        high = day.High
    #for zero division error handling
    if low == 0:
      delta_percent = 0
    else:
      delta_percent = 100 * (high - low) / low #check for division by 0
    Open = df_lookup(hist, 0, "Open")

    # some error handling: 
    if len(hist >= 5):
      Close = df_lookup(hist, 4, "Close")
    else :
      Close = Open

    if (Open == 0): #correct zero division error
      delta_price = 0
    else:
      delta_price = 100 * (Close - Open) / Open 
 
    print(stock+" "+str(delta_percent)+ " "+ str(delta_price))
    pair = [stock, delta_percent, delta_price]
    movement_list.append(pair)
    stock_writer.writerow(pair)
  return movement_list 

In [42]:
#test
start = time.perf_counter()
get_movement_list(df, "1d")
end = time.perf_counter()
print(end - start)

AAPL 2.87723097746304 -100.0
MSFT 2.2810133437697355 -100.0
0.5117874650004524


In [25]:
# time get movement list function #started 8:59, end  10:03

start = time.perf_counter()
get_movement_list(df, "1d")
end = time.perf_counter()
print(end - start)

AACG 9.929079773052568 -100.0
AACQ 0.9708682130449126 -100.0
AACQU 1.3501402055799134 -100.0
AACQW 9.223304004760964 -100.0
AAL 5.523011363568347 -100.0
AAME 5.665025180173393 -100.0
AAOI 3.327505137760572 -100.0
AAON 1.8694145259286057 -100.0
AAPL 2.87723097746304 -100.0
AAWW 2.704564539980032 -100.0
AAXJ 1.818381234871679 -100.0
ABCB 2.521769623700731 -100.0
ABCL 6.231115216816332 -100.0
ABCM 1.4982189763275577 -100.0
ABEO 8.181821038900265 -100.0
ABGI 0.5035169740577544 -100.0
ABIO 7.106592104181898 -100.0
ABMD 2.807116613641615 -100.0
ABNB 4.638965315369753 -100.0
ABST 3.081611331704584 -100.0
ABTX 2.866939793123955 -100.0
ABUS 6.329111122571895 -100.0
ACAC 2.5411073076067763 -100.0
ACACU 2.697492580772217 -100.0
ACACW 8.888893892735743 -100.0
ACAD 3.7324473348679055 -100.0
ACAHU 0.40200967457085374 -100.0
ACBI 3.186169855709594 -100.0
ACCD 10.311371223583093 -100.0
ACER 8.620688237701678 -100.0
ACET 7.382553084778064 -100.0
ACEV 1.4354122004059457 -100.0
ACEVU 4.378283683534454 -1

In [ ]:
def get_highest_movers(movement_list):
  volatile_stocks = []
  for entry in movementlist:
    if entry[1] > float(100):
      volatile_stocks.append(entry)
  return volatile_stocks

In [ ]:
get_movement_list(df, "1d")

In [4]:
yf.Ticker("GOOGL").history(period="2d")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-03-16,2065.989990,2113.679932,2059.290039,2083.889893,1592800,0,0
2021-03-17,2068.469971,2099.000000,2044.119995,2082.219971,1292400,0,0
